In [2]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
 
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import StratifiedKFold
 
###############################################################################
from sklearn import cross_validation, svm
from sklearn.externals import joblib
import pandas as pd
import func

datapath = '../data/'
data = pd.read_csv(datapath + 'data_gzip.csv.filter.mean')
info = pd.read_csv(datapath + 'info.csv')
merge = pd.merge(data, info, on='id')
with_pca = False

# 提取特征和y
labels = merge['nst_result']
merge.drop(['id', 'nst_result'], axis=1, inplace=True)

# 去掉info带来的特征
info_features = ['jianceshichang', 'jixian', 'jixianbianyi', 'taidongjiasutime', 'taidongjiasufudu', 'taidongcishu',
                 'bianyizhouqi', 'jiasu', 'jiasuzhenfu', 'jiasutimes', 'jiansu', 'jiansuzhenfu', 'jiansuzhouqi',
                 'jiansutimes', 'wanjiansu', 'wanjiansuzhenfu', 'wanjiansuzhouqi', 'wanjiansutimes', 'jixian',
                 'jixianbianyi', 'taidongjiasutime', 'taidongjiasufudu', 'taidongcishu']

merge.drop(info_features, axis=1, inplace=True)
train = merge.fillna(0).as_matrix()

lowDMat, reconMat = func.pca(train, 100)
trainMat = lowDMat if with_pca else train
X_train, X_test, y_train, y_test = cross_validation.train_test_split(trainMat, labels, test_size=0.3)

In [3]:
X = trainMat
y = labels
X, y = X[y != 4], y[y != 4]
X, y = X[y != 3], y[y != 3]
n_samples, n_features = X.shape

In [17]:
#X.shape

(24278, 1201)

In [15]:
###############################################################################
# Classification and ROC analysis
#分类，做ROC分析
 
# Run classifier with cross-validation and plot ROC curves
#使用5折交叉验证，并且画ROC曲线
cv = StratifiedKFold(y, n_folds=5)
classifier = svm.SVC(kernel='linear', probability=True,)
                     #random_state=random_state)#注意这里，probability=True,需要，不然预测的时候会出现异常。另外rbf核效果更好些。
 
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(cv):
	#通过训练数据，使用svm线性核建立模型，并对测试集进行测试，求出预测得分
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
#    print set(y[train])                     #set([0,1]) 即label有两个类别
#    print len(X[train]),len(X[test])        #训练集有84个，测试集有16个
#    print "++",probas_                      #predict_proba()函数输出的是测试集在lael各类别上的置信度，
#    #在哪个类别上的置信度高，则分为哪类
    # Compute ROC curve and area the curve
    #通过roc_curve()函数，求出fpr和tpr，以及阈值
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)			#对mean_tpr在mean_fpr处进行插值，通过scipy包调用interp()函数
    mean_tpr[0] = 0.0 								#初始处为0
    roc_auc = auc(fpr, tpr)
    #画图，只需要plt.plot(fpr,tpr),变量roc_auc只是记录auc的值，通过auc()函数能计算出来
    plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    print(i)
 
#画对角线
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
 
mean_tpr /= len(cv) 					#在mean_fpr100个点，每个点处插值插值多次取平均
mean_tpr[-1] = 1.0 						#坐标最后一个点为（1,1）
mean_auc = auc(mean_fpr, mean_tpr)		#计算平均AUC值
#画平均ROC曲线
#print mean_fpr,len(mean_fpr)
#print mean_tpr
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)
 
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').